In [4]:
import numpy as np
import xarray as xr
import xgcm
import dask

In [5]:
path = "/path/to/model/output/" 

### Compute heat flux
Compute the transient heat flux based on 5-day anomalies to a climatological seasonal cycle:  

$THT = \rho C_{p} \, v'T'$  

with
$v' = v - \langle v \rangle$ and $T' = T - \langle T \rangle$  
where $\langle \, \rangle$  indicates a seasonal climatology, $\rho=1035$ kg m$^{-3}$ is a reference density and $C_{p}=3994$ J$\,$kg$^{-1}\,$K$^{-1}$ is the specific heat capacity of seawater at constant pressure.

In [6]:
# definition of constants
rho = 1035.
Cp = 3994.

# define grid metrics for interpolation of T to u- and v-points
metrics = {
    ('X'): ['dxC', 'dxG', 'dxF', 'dxV'], # X distances
    ('Y'): ['dyC', 'dyG', 'dyF', 'dyU'], # Y distances
    ('Z'): ['drF', 'drW', 'drS', 'drC'], # Z distances
    ('X', 'Y'): ['rAw', 'rAs', 'rA', 'rAz'] # Areas in x-y plane
}

# compute seperately for each decade to get an idea of the internal variability across decades
for y in range(0,10):
    print(y)
    y1 = "02" + str(y) + "1"
    y2 = "0" + str(int(y1)+9)
    data = xr.open_zarr(path + "zarr_Diags/output.5d.zarr/").sel(time=slice(y1 + '-01-01', y2 + '-12-30'))
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        ubar = data.UVEL.groupby('time.month').mean("time")
        vbar = data.VVEL.groupby('time.month').mean("time")
        tbar = data.THETA.groupby('time.month').mean("time")
        uprime = data.UVEL.groupby('time.month') - ubar
        vprime = data.VVEL.groupby('time.month') - vbar
        tprime = data.THETA.groupby('time.month') - tbar
    grid = xgcm.Grid(data, periodic=["X"], metrics=metrics)
    THT = xr.Dataset()
    THT["THTu"] = rho * Cp * (uprime * grid.interp(tprime, "X") * data.hFacW)
    THT["THTv"] = rho * Cp * (vprime * grid.interp(tprime, "Y", boundary="fill", fill_value=0) * data.hFacS)
    THT.to_netcdf(path + "post/THT." + y1 + "_" + y2 + ".nc")

0
1
2
3
4
5
6
7
8
9
